### Preparing the Data

In [9]:
import requests
import json

url = 'https://api.thevirustracker.com/free-api?countryTimeline=US'

x = requests.post(url) 
Data= json.loads(x.content)['timelineitems'][0]
Data 

{'1/22/20': {'new_daily_cases': 1,
  'new_daily_deaths': 0,
  'total_cases': 1,
  'total_deaths': 0,
  'total_recoveries': 0},
 '1/23/20': {'new_daily_cases': 0,
  'new_daily_deaths': 0,
  'total_cases': 1,
  'total_deaths': 0,
  'total_recoveries': 0},
 '1/24/20': {'new_daily_cases': 1,
  'new_daily_deaths': 0,
  'total_cases': 2,
  'total_deaths': 0,
  'total_recoveries': 0},
 '1/25/20': {'new_daily_cases': 0,
  'new_daily_deaths': 0,
  'total_cases': 2,
  'total_deaths': 0,
  'total_recoveries': 0},
 '1/26/20': {'new_daily_cases': 3,
  'new_daily_deaths': 0,
  'total_cases': 5,
  'total_deaths': 0,
  'total_recoveries': 0},
 '1/27/20': {'new_daily_cases': 0,
  'new_daily_deaths': 0,
  'total_cases': 5,
  'total_deaths': 0,
  'total_recoveries': 0},
 '1/28/20': {'new_daily_cases': 0,
  'new_daily_deaths': 0,
  'total_cases': 5,
  'total_deaths': 0,
  'total_recoveries': 0},
 '1/29/20': {'new_daily_cases': 0,
  'new_daily_deaths': 0,
  'total_cases': 5,
  'total_deaths': 0,
  'total_r

In [19]:
import pandas as pd

data= pd.DataFrame(Data)
data

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/01/20,2/02/20,2/03/20,2/04/20,2/05/20,2/06/20,2/07/20,2/08/20,2/09/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/01/20,...,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/01/20,6/02/20,6/03/20,6/04/20,6/05/20,6/06/20,6/07/20,6/08/20,6/09/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/01/20,7/02/20,7/03/20,stat
new_daily_cases,1,0,1,0,3,0,0,0,0,2,1,0,3,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,2,0,0,36,0,6,1,2,8,6,...,22860,17158,22396,24403,21995,20340,21698,21775,19494,20596,24346,22578,21762,17685,17276,25510,19562,25697,27210,21553,17456,24182,24473,26427,32584,31783,25079,30928,35385,34631,40464,45330,42435,39677,37370,46552,50111,54437,55756,ok
new_daily_deaths,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,841,1396,1291,1147,976,634,684,1117,973,958,949,702,780,452,817,1082,867,913,734,382,361,764,809,671,727,584,304,402,806,745,648,2410,512,296,321,1197,684,681,655,ok
total_cases,1,1,2,2,5,5,5,5,5,7,8,8,11,11,11,11,11,11,11,11,12,12,13,13,13,13,13,13,13,13,15,15,15,51,51,57,58,60,68,74,...,1680301,1697459,1719855,1744258,1766253,1786593,1808291,1830066,1849560,1870156,1894502,1917080,1938842,1956527,1973803,1999313,2018875,2044572,2071782,2093335,2110791,2134973,2159446,2185873,2218457,2250240,2275319,2306247,2341632,2376263,2416727,2462057,2504492,2544169,2581539,2628091,2678202,2732639,2788395,ok
total_recoveries,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,3,3,3,3,3,3,3,3,3,3,3,5,5,5,5,6,6,6,7,7,7,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ok
total_deaths,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,98875,100271,101562,102709,103685,104319,105003,106120,107093,108051,109000,109702,110482,110934,111751,112833,113700,114613,115347,115729,116090,116854,117663,118334,119061,119645,119949,120351,121157,121902,122550,124960,125472,125768,126089,127286,127970,128651,129306,ok


In [47]:
data= pd.DataFrame(Data)
series = data.iloc[0].to_numpy()
series = series[:-1]
series = series.astype('float32')
mean= np.mean(series)
var= np.var(series)
series = (series - mean)/var
series

array([-8.21917565e-05, -8.21965878e-05, -8.21917565e-05, -8.21965878e-05,
       -8.21820868e-05, -8.21965878e-05, -8.21965878e-05, -8.21965878e-05,
       -8.21965878e-05, -8.21869253e-05, -8.21917565e-05, -8.21965878e-05,
       -8.21820868e-05, -8.21965878e-05, -8.21965878e-05, -8.21965878e-05,
       -8.21965878e-05, -8.21965878e-05, -8.21965878e-05, -8.21965878e-05,
       -8.21917565e-05, -8.21965878e-05, -8.21917565e-05, -8.21965878e-05,
       -8.21965878e-05, -8.21965878e-05, -8.21965878e-05, -8.21965878e-05,
       -8.21965878e-05, -8.21965878e-05, -8.21869253e-05, -8.21965878e-05,
       -8.21965878e-05, -8.20225541e-05, -8.21965878e-05, -8.21675858e-05,
       -8.21917565e-05, -8.21869253e-05, -8.21579160e-05, -8.21675858e-05,
       -8.20805653e-05, -8.20999048e-05, -8.20467249e-05, -8.18678527e-05,
       -8.19790439e-05, -8.15197709e-05, -8.16358006e-05, -8.18823537e-05,
       -8.03788498e-05, -8.06399112e-05, -8.03498478e-05, -7.97020330e-05,
       -7.95473316e-05, -

In [48]:
split_time = int(0.8 * time.shape[0])
#time_train = time[:split_time]
x_train = series[:split_time]
#time_valid = time[split_time:]
x_valid = series[split_time:]

window_size = 10
batch_size = 5
shuffle_buffer_size = 500

In [31]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [32]:
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

### The model

In [50]:
import tensorflow as tf
import numpy as np


tf.keras.backend.clear_session()
tf.random.set_seed(51)
np.random.seed(51)
#window_size = 64
#batch_size = 256
train_set = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)
print(train_set)
print(x_train.shape)

model = tf.keras.models.Sequential([
  
  tf.keras.layers.Conv1D(filters=32, kernel_size=5,
                      strides=1, padding="causal",
                      activation="relu",
                      input_shape=[None, 1]),
  tf.keras.layers.LSTM(64, return_sequences=True),
  tf.keras.layers.LSTM(64, return_sequences=True),
  tf.keras.layers.Dense(30, activation="relu"),
  tf.keras.layers.Dense(10, activation="relu"),
  tf.keras.layers.Dense(1),
  #tf.keras.layers.Lambda(lambda x: x * 400)
])

lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))
optimizer = tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
history = model.fit(train_set, epochs=100, callbacks=[lr_schedule])
test_set = windowed_dataset(x_valid, window_size, batch_size, shuffle_buffer_size)
model.evaluate(test_set)

<PrefetchDataset shapes: ((None, None, 1), (None, None, 1)), types: (tf.float32, tf.float32)>
(131,)
Epoch 1/100
25/25 [==============================] - 0s 13ms/step - loss: 2.1104e-09 - mae: 5.9323e-05 - lr: 1.0000e-08
Epoch 2/100
25/25 [==============================] - 0s 13ms/step - loss: 2.1104e-09 - mae: 5.9322e-05 - lr: 1.1220e-08
Epoch 3/100
25/25 [==============================] - 0s 13ms/step - loss: 2.1104e-09 - mae: 5.9322e-05 - lr: 1.2589e-08
Epoch 4/100
25/25 [==============================] - 0s 13ms/step - loss: 2.1104e-09 - mae: 5.9322e-05 - lr: 1.4125e-08
Epoch 5/100
25/25 [==============================] - 0s 12ms/step - loss: 2.1104e-09 - mae: 5.9322e-05 - lr: 1.5849e-08
Epoch 6/100
25/25 [==============================] - 0s 14ms/step - loss: 2.1103e-09 - mae: 5.9322e-05 - lr: 1.7783e-08
Epoch 7/100
25/25 [==============================] - 0s 12ms/step - loss: 2.1103e-09 - mae: 5.9322e-05 - lr: 1.9953e-08
Epoch 8/100
25/25 [==============================] - 0s 11m

[3.147857530194642e-09, 6.74490220262669e-05]